In [2]:
import json
import pandas
from datetime import datetime
from pymongo import MongoClient
from bson import json_util
from collections import Counter

#### Подключение к БД:

In [3]:
client = MongoClient("localhost:27017")
db = client['auto-programmer-db']
firmware_collection = db['firmware_projects_arch']
collection_list = db.list_collection_names()

if collection_list is not None:
    print("Подключение к MongoDB успешно")
else:
    print("Не удалось подключиться к MongoDB.")

Подключение к MongoDB успешно


#### Поиск документа по запросу:

In [4]:
def find_documents(query:dict):
    """
    Функция для поиска документов по заданному запросу.
    
    Parameters:
        query(dict): объект типа словарь, содержащий условие поиска
        
    Returns:
        json_documents: итерируемый объект JSON, содержащий документы, удовлетворяющие запросу
        
    Example_usage: 
        find_documents({"project_name":"td_ble_523"})
    """
    cursor = list(firmware_collection.find(query))
    if cursor:
        json_documents = json_util.dumps(cursor)
        return json.loads(json_documents)
        return json_documents
    else:
        return 'Документы не найдены'

Получим документ:

In [5]:
my_document = find_documents({"project_name":"td_ble_523"})

#### Получение списка уникальных имен

In [6]:
def find_unique_project_names(collection, timestamp:str = None):
    """
    Функция для поиска уникальных имен проектов.
    
    Parameters:
        firmware_collection(pymongo.collection.Collection): Коллекция MongoDB, из которой нужно извлечь данные. 
        timestamp(str, optional): опциональный параметр, при наличии которого будет фильтрация только за указанную дату
                        (по умолчанию за все время)
        
    Returns:
       unique_project_names(list): возвращает список с уникальными именами проектов
        
    Example_usage: 
        find_unique_project_names(firmware_collection)
        find_unique_project_names(firmware_collection, '2024-03-21')
        
    """
    query = {}
    if timestamp:
        query['timestamp'] = {'$regex': timestamp}

    unique_project_names = firmware_collection.distinct('project_name', query)
    return unique_project_names

Получим список уникальных имен:

In [7]:
find_unique_project_names(firmware_collection, '2024-03-21')

['td_ble_523']

#### Функция для возврата словаря mac:serial в заданном промежутке времени

In [8]:
def get_serial_mac_pairs(collection, start_date, end_date):
    """
    Функция для извлечения списка пар серийный номер - MAC-адрес за указанный промежуток времени.

    Parameters:
        collection (pymongo.collection.Collection): Коллекция MongoDB, из которой нужно извлечь данные. 
        start_date (str): Начальная дата в формате 'YYYY-MM-DD'.
        end_date (str): Конечная дата в формате 'YYYY-MM-DD'.
        
    Returns:
       serial_mac_pairs (list): Список, который содержит пары значений serial_number:mac_address.
       
    Example_usage:
        get_serial_mac_pairs(firmware_collection, "2024-03-20", "2024-03-24")

    """
    query = {'timestamp': {'$gte': start_date,
                           '$lte': end_date}}
    projection = {'_id': 0, 'plates': 1}

    cursor = collection.find(query, projection)

    serial_mac_pairs = []
    for document in cursor:
        plates = document.get('plates', {})
        for plate_info in plates.values():
            serial_number = plate_info.get('serial_number')
            mac_address = plate_info.get('mac_address')
            serial_mac_pairs.append((serial_number, mac_address)) 
    if serial_mac_pairs:
        return serial_mac_pairs
    else:
        return {'Документы за выбранные даты найдены'}

Получим список серийников и mac-адресов:

In [9]:
print(get_serial_mac_pairs(firmware_collection, "2024-03-22", "2024-03-23"))

[(100153, 'A7:D6:8E:73:86:C5 C5:86:73:E'), (None, None), (None, None), (None, None), (None, None), (None, None), (None, None), (None, None), (None, None), (None, None), (None, None), (None, None), (None, None), (None, None), (None, None), (None, None), (None, None), (None, None), (None, None), (100154, '04:44:3A:65:A0:DE DE:A0:65:A'), (100155, 'A7:D6:8E:73:86:C5 C5:86:73:E'), (100156, '1C:0C:49:B7:0E:C0 C0:0E:B7:9'), (100157, 'F7:9E:69:40:2A:C5 C5:2A:40:9'), (None, None), (None, None), (None, None), (None, None), (None, None), (None, None), (None, None), (None, None), (None, None), (None, None), (None, None), (None, None), (None, None), (None, None), (None, None), (None, None), (None, None), (100157, 'A7:D6:8E:73:86:C5 C5:86:73:E'), (100158, '1C:0C:49:B7:0E:C0 C0:0E:B7:9'), (100159, 'F7:9E:69:40:2A:C5 C5:2A:40:9'), (100160, '26:64:E1:B6:15:FF FF:15:B6:1'), (100161, '06:80:D5:FD:CE:DA DA:CE:FD:5'), (None, None), (None, None), (None, None), (None, None), (None, None), (None, None), (None

#### Функция для возврата словаря со статусами в заданном промежутке времени и проекте

In [10]:
def get_project_statuses(collection, start_date, end_date, project_name):
    """
    Функция для получения словаря со статусами по проекту в заданный промежуток времени.

    Parameters:
        collection (pymongo.collection.Collection): Коллекция MongoDB, из которой нужно извлечь данные. 
        start_date (str): Начальная дата в формате 'YYYY-MM-DD'.
        end_date (str): Конечная дата в формате 'YYYY-MM-DD'.
        project_name(str): Имя проекта
        
    Returns:
       project_status_dict (dict): Cловарь, который содержит статусы и информацию о них.
       
    Example_usage:
        get_project_statuses(firmware_collection, "2024-03-20", "2024-03-24", "td_ble_523")

    """
    query = {'project_name' : project_name,
             'timestamp' : {'$gte': start_date,
                           '$lte': end_date}}
    projection = {'_id': 0, 'plates': 1}

    cursor = collection.find(query, projection)

    project_status_dict = {}
    statuses_list = []
    for document in cursor:
        plates = document.get('plates', {})
        for plate_info in plates.values():
            statuses = plate_info.get('status')
            statuses_list.append(statuses) 
    if statuses_list:
        status_counts = Counter(statuses_list)
        project_status_dict = {'total': len(statuses_list)}
        project_status_dict.update(status_counts)    
        return project_status_dict
    else:
        return f'Плат со статусом в проекте {project_name} за выбранный период времени не найдено'

Получим информацию о статусах плат проекта:

In [11]:
print(get_project_statuses(firmware_collection, "2024-03-22", "2024-03-23", "td_ble_523"))

{'total': 80, 'success': 10, 'nottodo': 68, 'fail': 2}


#### Функция для возврата списка серийников в проекте с заданным статусом за промежуток времени

In [12]:
def get_serial_by_project_status(collection, start_date, end_date, project_name, status):
    """
    Функция для возврата списка серийников в проекте с заданным статусом за промежуток времени

    Parameters:
        collection (pymongo.collection.Collection): Коллекция MongoDB, из которой нужно извлечь данные. 
        start_date (str): Начальная дата в формате 'YYYY-MM-DD'.
        end_date (str): Конечная дата в формате 'YYYY-MM-DD'.
        project_name(str): Имя проекта
        status(str): Статус платы
        
    Returns:
       serial_number_list (list): Лист содержащий серийники плат с заданным статусом
       
    Example_usage:
        get_serial_by_project_status(firmware_collection, "2024-03-22", "2024-03-24", "td_ble_523", "success")

    """
    query = {'project_name' : project_name,
             'timestamp' : {'$gte': start_date,
                           '$lte': end_date}}
    projection = {'_id': 0, 'plates': 1}

    cursor = collection.find(query, projection)

    serial_number_list = []
    for document in cursor:
        plates = document.get('plates', {})
        for plate_info in plates.values():
            plate_status = plate_info.get('status')
            if plate_status == status:
                serial_number = plate_info.get('serial_number')
                serial_number_list.append(serial_number)
        return serial_number_list
    else:
        return f'Плат со статусом {status} в проекте {project_name} за выбранный период времени не найдено'

Получим серийники плат:

In [13]:
get_serial_by_project_status(firmware_collection, "2024-03-22", "2024-03-24", "td_ble_523", "success")

[100153, 100154]

#### Функция для возврата суммы времени, затраченного на прошивку плат одного типа

In [14]:
def project_total_time (collection, start_date, end_date, project_name):
    '''
    Функция для возврата суммы времени, затраченного на прошивку платы одного типа

    Parameters:
        collection (pymongo.collection.Collection): Коллекция MongoDB, из которой нужно извлечь данные. 
        start_date (str): Начальная дата в формате 'YYYY-MM-DD'.
        end_date (str): Конечная дата в формате 'YYYY-MM-DD'.
        project_name(str): Имя проекта
        
    Returns:
       total_seconds (float): сумма времени 
       
    Example_usage:
        project_total_time(firmware_collection, "2024-03-22", "2024-03-24", "td_ble_523")
    '''
    query = {'project_name' : project_name,
             'timestamp' : {'$gte': start_date,
                           '$lte': end_date}}
    projection = {'_id': 0, 'cycle_time': 1}
    
    cursor = collection.find(query, projection)
    project_time = 0
    for record in cursor:
        project_time += record.get('cycle_time', 0)

    return project_time

Посчитаем время, ушедшее на прошивку проекта:

In [15]:
project_time = project_total_time(firmware_collection, "2024-03-22", "2024-03-24", "td_ble_523")
print(f'На прошивку проектов td_ble_523 ушло {round(project_time, 2)} секунд')

На прошивку проектов td_ble_523 ушло 269.71 секунд
